In [1]:
import pandas as pd

# Criando uma pivot table usando o método `pivot` e `pivot_table`
- Uma pivot table é interessante para agregar informações de uma tabela que repete valores multiplas vezes
- É uma maneira de comprimir os dados e as informações
- vamos carregar um dataset que tem esse comportamento, ele vai repetir as vendas de vendedores ao longo das datas:

In [2]:
salesmen = pd.read_csv("../data/salesmen.csv", parse_dates=["Date"])
salesmen.head()

/tmp/ipykernel_79438/2510303856.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  salesmen = pd.read_csv("../data/salesmen.csv", parse_dates=["Date"])


,Date,Salesman,Revenue
0,2016-01-01,Bob,7172
1,2016-01-02,Bob,6362
2,2016-01-03,Bob,5982
3,2016-01-04,Bob,7917
4,2016-01-05,Bob,7837


- Primeiro, vamos criar um novo dataframe com o método `pivot`

In [3]:
pivot_df = salesmen.pivot(index="Date", columns="Salesman", values="Revenue")
pivot_df.head()

Salesman,Bob,Dave,Jeb,Oscar,Ronald
Date,,,,,
2016-01-01,7172,1864,4430,5250,2639
2016-01-02,6362,8278,8026,8661,4951
2016-01-03,5982,4226,5188,7075,2703
2016-01-04,7917,3868,3144,2524,4258
2016-01-05,7837,2287,938,2793,7771


- Observe que agora nós temos uma nova tabela em que os vendedores se tornaram colunas. Nao tem mais centenas de repetição

## Para exemplificar o Pivot Table, vamos carregar outro dataset
- A ideia é como é feito no excel: pegar os dados de entradas do DF e fazer uma agregação

In [4]:
foods = pd.read_csv("../data/foods.csv")
foods.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/foods.csv'

- Vamos verificar qual o gasto médio por gênero:

In [7]:
foods.pivot_table(values="Spend", index="Gender", aggfunc="mean")

,Spend
Gender,
Female,50.709629
Male,49.397623


- Agora, qual a soma total do gasto por genero:

In [8]:
foods.pivot_table(values="Spend", index="Gender", aggfunc="sum")

,Spend
Gender,
Female,25963.33
Male,24106.04


- Podemos trocar o indice tranquilamente. Por exemplo, vamos usar o `Item`:

In [11]:
foods.pivot_table(values="Spend", index="Item", aggfunc="sum")

,Spend
Item,
Burger,7765.73
Burrito,8270.44
Chalupa,7644.52
Donut,8758.76
Ice Cream,8886.99
Sushi,8742.93


- Ou, podemos usar multiplos indices:

In [12]:
foods.pivot_table(values="Spend", index=["Gender", "Item"], aggfunc="sum")

Spend
Gender Item              
Female Burger     4094.30
       Burrito    4257.82
       Chalupa    4152.26
       Donut      4743.00
       Ice Cream  4032.87
       Sushi      4683.08
Male   Burger     3671.43
       Burrito    4012.62
       Chalupa    3492.26
       Donut      4015.76
       Ice Cream  4854.12
       Sushi      4059.85

- Ainda mais complexo, podemos calcular o gasto médio, considerando tanto genero quanto item, porém estratificando por cidade:
    - Perceba que essa é uma query complexa que é solucionada de maneira rapida e facil com pandas
    - `aggfunc` pode assumir, por exemplo: `mean`, `count`, `max`, `min`, `sum`...

In [14]:
foods.pivot_table(values="Spend", index=["Gender", "Item"], columns="City", aggfunc="mean")

City               New York  Philadelphia   Stamford
Gender Item                                         
Female Burger     51.626667     52.878710  45.037778
       Burrito    42.563043     52.098571  53.532647
       Chalupa    46.135789     52.291563  64.094000
       Donut      46.670323     54.642000  48.734118
       Ice Cream  56.356296     46.225625  46.910455
       Sushi      47.751290     58.096000  45.622187
Male   Burger     58.822273     44.675238  46.424516
       Burrito    55.976000     43.764333  46.438929
       Chalupa    49.110800     48.444783  50.011304
       Donut      44.842333     37.859394  49.004483
       Ice Cream  55.297586     53.445610  42.368800
       Sushi      51.709259     49.852857  70.434444

- Ainda mais complexo, podemos ter multiplas colunas:

In [15]:
foods.pivot_table(values="Spend", index=["Gender", "Item"], columns=["Frequency", "City"], aggfunc="mean")

Frequency             Daily                            Monthly               \
City               New York Philadelphia   Stamford   New York Philadelphia   
Gender Item                                                                   
Female Burger     43.778333    77.226667  48.220000  57.286667    53.762500   
       Burrito    44.890000    53.595000  39.126000  40.913333    17.140000   
       Chalupa    43.190000    23.490000  95.700000  79.185000    72.490000   
       Donut      39.841667    61.850000  41.450000  71.132500    50.250000   
       Ice Cream  65.547500    59.230000  46.440000  46.265000    37.255000   
       Sushi      40.535000    58.088333  56.181429  46.580000    78.710000   
Male   Burger     63.892000    37.566667  49.430000  62.430000    71.046667   
       Burrito    78.736667    41.440000  69.057500  49.180000    29.860000   
       Chalupa    27.045000    68.702500  48.160000  66.752000    45.350000   
       Donut      46.000000    47.677500  64.710000  45.932500    51.858000   
       Ice Cream  59.406667    29.738000  15.170000  50.733333    64.425000   
       Sushi      56.392500    46.821667        NaN  53.857500    77.795000   

Frequency                        Never                              Often  \
City               Stamford   New York Philadelphia   Stamford   New York   
Gender Item                                                                 
Female Burger     59.622500  97.890000    54.742500  45.485000  23.740000   
       Burrito    67.940000  47.432500    63.716667  52.334286  34.533333   
       Chalupa    80.990000  35.150000    30.492500  52.120000  39.730000   
       Donut      45.860000  56.070000    72.263333  52.443333  32.657500   
       Ice Cream  41.950000  68.716667    39.007500  77.660000  58.065000   
       Sushi      54.195000  69.330000    47.645000  19.560000  49.134286   
Male   Burger     13.580000  90.320000     8.655000        NaN  27.735000   
       Burrito    39.866667  28.926667    47.290000  70.368000  47.480000   
       Chalupa    57.293333  39.818000    48.596000  46.233333  62.880000   
       Donut      29.882500  43.926000    26.825000  54.910000  46.600000   
       Ice Cream  74.360000  28.770000    80.485000  39.923333  69.510000   
       Sushi      93.030000  54.670000    40.961667  68.300000   9.310000   

Frequency         ...       Once     Seldom                            Weekly  \
City              ...   Stamford   New York Philadelphia   Stamford  New York   
Gender Item       ...                                                           
Female Burger     ...  31.683333  31.580000    58.435714  48.765000  92.17500   
       Burrito    ...  56.003333  83.770000    49.527500  78.163333  13.23000   
       Chalupa    ...  40.590000  40.000000    54.902000  58.416667  42.88000   
       Donut      ...  79.120000  30.270000    45.812500  34.886667  71.39000   
       Ice Cream  ...  55.866000  80.783333    50.775000  58.865000  56.90500   
       Sushi      ...  38.950000  87.700000    62.848000  27.820000  51.36125   
Male   Burger     ...  36.293333  75.226667    47.015000  53.250000  69.69000   
       Burrito    ...  15.075000  67.466667    27.710000   9.840000  64.18500   
       Chalupa    ...        NaN  11.690000    65.375000  34.804000  54.40000   
       Donut      ...  27.978000  16.250000    33.003333  40.827500  37.22000   
       Ice Cream  ...  36.542000  67.605000    69.220000  20.415000  44.77200   
       Sushi      ...  62.930000  53.496667    91.260000  74.716667  50.05000   

Frequency                                  Yearly                          
City             Philadelphia Stamford   New York Philadelphia   Stamford  
Gender Item                                                                
Female Burger       16.000000  31.0040  64.825000    61.585000  51.171667  
       Burrito      31.410000  46.1820  35.630000    38.916667  43.245000  
       Chalupa      28.136667  68.2300  52.606667    56.048889

# Usando o método `melt()`
- Basicamente é a operação inversa do `pivot_table()`
- Pega um dado agregado e transforma e uma tabela

In [5]:
sales = pd.read_csv("../data/quarters.csv")
sales.head()

,Salesman,Q1,Q2,Q3,Q4
0,Boris,602908,233879,354479,32704
1,Bob,43790,514863,297151,544493
2,Tommy,392668,113579,430882,247231
3,Travis,834663,266785,749238,570524
4,Donald,580935,411379,110390,651572


- basicamente, vamos mover as colunas Q1-4 para virar linhas da tabela:

In [6]:
melt_sales = pd.melt(sales, id_vars="Salesman")
melt_sales

,Salesman,variable,value
0,Boris,Q1,602908
1,Bob,Q1,43790
2,Tommy,Q1,392668
3,Travis,Q1,834663
4,Donald,Q1,580935
5,Ted,Q1,656644
6,Jeb,Q1,486141
7,Stacy,Q1,479662
8,Morgan,Q1,992673
9,Boris,Q2,233879


- Perceba que `variable` são as colunas que queremos expandir e `values` são os valores que cada uma delas assume
- Podemos, obviamente, controlar os nomes delas:

In [20]:
melt_sales = pd.melt(sales, id_vars="Salesman", var_name="Quarter", value_name="Revenue")
melt_sales.head()

,Salesman,Quarter,Revenue
0,Boris,Q1,602908
1,Bob,Q1,43790
2,Tommy,Q1,392668
3,Travis,Q1,834663
4,Donald,Q1,580935
